## ML-第六版本-类别特征编码-时间序列特征-交叉累计特征-验证版
+ 该版本用于验证woe编码的问题，由于用了category_encoders的woe编码函数，使得auc特别高，但之前用我自己版本的woe编码效果却没那么好，所以考虑用我自己的woe编码版本试试

In [2]:
import os
import re
import numpy as np
import pandas as pd
import datetime
import pickle as pkl

import matplotlib.pyplot as plt
import seaborn as sns

import scorpyo as sp
from null_importance import get_null_importance
from time_sequence_feats import get_time_base, get_sequence_statis, get_sequence_groupby_statis
import category_encoders as ce

from gensim.models import word2vec

import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold

import warnings
warnings.filterwarnings("ignore")

pd.set_option('max_rows', 320, 'max_columns',100)

In [3]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')
path_results_version  = os.path.join(path_results, 'version_2')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')

## results
path_output_report = os.path.join(path_results, '01_原始数据探察_20221014.xlsx')

y_label = "is_risk"


# new_train_data
path_new_train_cate = os.path.join(path_new_data, 'train_cate.pkl')
path_new_test_cate  = os.path.join(path_new_data, 'test_cate.pkl')

path_new_train_time_sequence = os.path.join(path_new_data, 'train_time_sequence.pkl')
path_new_test_time_sequence  = os.path.join(path_new_data, 'test_time_sequence.pkl')

path_new_train_time_sequence_woe = os.path.join(path_new_data, 'train_time_sequence_woe.pkl')
path_new_test_time_sequence_woe  = os.path.join(path_new_data, 'test_time_sequence_woe.pkl')

path_new_train_cumsum = os.path.join(path_new_data, 'train_cumsum.pkl')
path_new_test_cumsum = os.path.join(path_new_data, 'test_cumsum.pkl')

path_new_train_cumsum_woe = os.path.join(path_new_data, 'train_cumsum_woe.pkl')
path_new_test_cumsum_woe = os.path.join(path_new_data, 'test_cumsum_woe.pkl')

path_new_train_embeding = os.path.join(path_new_data, 'train_embeding.pkl')
path_new_test_embeding = os.path.join(path_new_data, 'test_embeding.pkl')

path_new_train_time_sequence_sp_woe = os.path.join(path_new_data, 'train_time_sequence_sp_woe.pkl')
path_new_test_time_sequence_sp_woe  = os.path.join(path_new_data, 'test_time_sequence_sp_woe.pkl')

path_new_train_cumsum_sp_woe = os.path.join(path_new_data, 'train_cumsum_sp_woe.pkl')
path_new_test_cumsum_sp_woe = os.path.join(path_new_data, 'test_cumsum_sp_woe.pkl')


path_new_train_sp_cate = os.path.join(path_new_data, 'train_sp_cate.pkl')
path_new_test_sp_cate  = os.path.join(path_new_data, 'test_sp_cate.pkl')

In [5]:
df_row_train = pd.read_csv(path_train)
df_row_val  = pd.read_csv(path_test)

## 工具函数

In [4]:
def save_pkl(data, path_data):
    """将数据保存为pkl"""
    with open(path_data, 'wb') as f:
        pkl.dump(data, f)
        
def read_pkl(path_data):
    """读取pkl格式数据"""
    with open(path_data, 'rb') as f:
        data = pkl.load(f)
    return data

## 数据读取

### 类别特征+时间序列特征+累计交叉特征

In [7]:
df_train_cate = read_pkl(path_new_train_cate)
df_val_cate = read_pkl(path_new_test_cate)

df_train_time_sequence = read_pkl(path_new_train_time_sequence)
df_val_time_sequence = read_pkl(path_new_test_time_sequence)

df_train_cumsum = read_pkl(path_new_train_cumsum)
df_val_cumsum = read_pkl(path_new_test_cumsum)

df_raw_train = pd.read_csv(path_train)
df_raw_val  = pd.read_csv(path_test)

## 时间序列特征woe编码+lightgbm

### 时间序列特征woe编码

In [8]:

df_row_train = pd.merge(left=df_train_time_sequence, right=df_raw_train[['id', y_label]], on='id')
df_row_val = pd.merge(left=df_val_time_sequence, right=df_raw_val[['id']], on='id')

In [12]:
bins_time_sequence = sp.woebin(df_row_train, y_label, var_skip=['id'])
bins_time_sequence = sp.bins_adj_nan_others(df_row_train, bins_time_sequence)

train_time_sequence_woe = sp.woebin_ply(df_row_train, bins_time_sequence)
val_time_sequence_woe  = sp.woebin_ply(df_row_val, bins_time_sequence)



There are 30 columns have only one unique values, which are removed from input dataset. 
 (ColumnNames: op_datetime_year, op_datetime_is_year_start, kurt_3_system_op_diff_second1, kurt_ip_transform_system_op_diff_second1_3, kurt_user_name_system_op_diff_second1_3, kurt_device_num_transform_system_op_diff_second1_3, kurt_department_system_op_diff_second1_3, kurt_browser_version_system_op_diff_second1_3, kurt_browser_system_op_diff_second1_3, kurt_os_type_system_op_diff_second1_3, kurt_os_version_system_op_diff_second1_3, kurt_ip_type_system_op_diff_second1_3, kurt_http_status_code_system_op_diff_second1_3, kurt_op_city_system_op_diff_second1_3, kurt_log_system_transform_system_op_diff_second1_3, kurt_url_system_op_diff_second1_3, kurt_3_system_op_diff_second2, kurt_ip_transform_system_op_diff_second2_3, kurt_user_name_system_op_diff_second2_3, kurt_device_num_transform_system_op_diff_second2_3, kurt_department_system_op_diff_second2_3, kurt_browser_version_system_op_diff_second2_3, kurt

[INFO] converting into woe values ...
Woe transformating on 47660 rows and 977 columns in 00:01:42
[INFO] converting into woe values ...
Woe transformating on 25710 rows and 977 columns in 00:00:56


In [14]:
feats = [col for col in train_time_sequence_woe.columns if col[-4:]=='_woe']
save_pkl(train_time_sequence_woe[feats+['id']], path_new_train_time_sequence_sp_woe)
save_pkl(val_time_sequence_woe[feats+['id']], path_new_test_time_sequence_sp_woe)

### lightgbm

In [17]:
df_train = train_time_sequence_woe.reset_index(drop=True)
df_test  = val_time_sequence_woe.reset_index(drop=True)

In [18]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [19]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [20]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.939603	valid_1's auc: 0.939006
[20]	training's auc: 0.942107	valid_1's auc: 0.941446
[30]	training's auc: 0.943311	valid_1's auc: 0.941004
[40]	training's auc: 0.946704	valid_1's auc: 0.941763
[50]	training's auc: 0.95392	valid_1's auc: 0.941707
[60]	training's auc: 0.962502	valid_1's auc: 0.940969
[70]	training's auc: 0.968816	valid_1's auc: 0.940761
[80]	training's auc: 0.974214	valid_1's auc: 0.940598
[90]	training's auc: 0.978991	valid_1's auc: 0.940413
[100]	training's auc: 0.981927	valid_1's auc: 0.939528
[110]	training's auc: 0.984616	valid_1's auc: 0.939197
[120]	training's auc: 0.987216	valid_1's auc: 0.938578
[130]	training's auc: 0.98864	valid_1's auc: 0.938121
Early stopping, best iteration is:
[34]	training's auc: 0.946345	valid_1's auc: 0.943493
----------- 1
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.936901	valid_1's auc: 0.936503
[

,name,importance
823,max_6_system_op_diff_second1_woe,66272.947554
181,avg_ip_transform_system_op_diff_second1_3_woe,40556.744302
277,max_ip_transform_system_op_diff_second2_3_woe,27316.857482
305,std_ip_transform_system_op_diff_second2_3_woe,5166.237092
243,std_ip_type_system_op_diff_second1_6_woe,4747.472191
672,avg_6_system_op_diff_second1_woe,4592.525234
535,avg_ip_transform_system_op_diff_second2_3_woe,4287.715193
140,op_datetime_hour_cos_woe,4140.732686
517,max_ip_transform_system_op_diff_second1_3_woe,2887.151223
645,median_ip_transform_system_op_diff_second1_3_woe,2539.646738


训练集AUC均值：0.9422815567850643, 训练集合AUC方差: 0.003257556375441052


## 交叉累计特征woe编码+lightgbm

### 交叉累计特征woe编码

In [22]:
df_row_train = pd.merge(left=df_train_cumsum, right=df_raw_train[['id', y_label]], on='id')
df_row_val = pd.merge(left=df_val_cumsum, right=df_raw_val[['id']], on='id')

In [23]:
bins_time_cumsum = sp.woebin(df_row_train, y_label, var_skip=['id'])
bins_time_cumsum = sp.bins_adj_nan_others(df_row_train, bins_time_cumsum)

train_time_cumsum_woe = sp.woebin_ply(df_row_train, bins_time_cumsum)
val_time_cumsum_woe  = sp.woebin_ply(df_row_val, bins_time_cumsum)

There are 147 columns have only one unique values, which are removed from input dataset. 
 (ColumnNames: system_system_op_times_groups_ip_transform_cumunique, system_system_op_times_groups_device_num_transform_cumunique, system_system_op_times_groups_department_nan_cumsum, system_system_op_times_groups_department_nan_error_code_cumsum, system_system_op_times_groups_browser_version_cumunique, system_system_op_times_groups_browser_cumunique, system_system_op_times_groups_os_type_cumunique, system_system_op_times_groups_os_version_cumunique, system_system_op_times_groups_ip_type_nan_cumsum, system_system_op_times_groups_ip_type_nan_error_code_cumsum, system_system_op_times_groups_http_status_code_cumunique, system_system_op_times_groups_op_city_cumunique, system_system_op_times_groups_log_system_transform_nan_cumsum, system_system_op_times_groups_log_system_transform_nan_error_code_cumsum, system_system_op_times_groups_url_cumunique, system_op_days_ip_transform_cumunique, system_op_days_d

[INFO] converting into woe values ...
Woe transformating on 47660 rows and 1680 columns in 00:02:45
[INFO] converting into woe values ...
Woe transformating on 25710 rows and 1680 columns in 00:01:27


In [24]:
feats = [col for col in train_time_cumsum_woe.columns if col[-4:]=='_woe']
save_pkl(train_time_cumsum_woe[feats+['id']], path_new_train_cumsum_sp_woe)
save_pkl(val_time_cumsum_woe[feats+['id']], path_new_test_cumsum_sp_woe)

### lightgbm

In [25]:
df_train = train_time_cumsum_woe.reset_index(drop=True)
df_test  = val_time_cumsum_woe.reset_index(drop=True)

In [26]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [27]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [28]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.934964	valid_1's auc: 0.932963
[20]	training's auc: 0.938009	valid_1's auc: 0.933679
[30]	training's auc: 0.940969	valid_1's auc: 0.93529
[40]	training's auc: 0.944641	valid_1's auc: 0.934179
[50]	training's auc: 0.947725	valid_1's auc: 0.933012
[60]	training's auc: 0.951484	valid_1's auc: 0.932714
[70]	training's auc: 0.958071	valid_1's auc: 0.933393
[80]	training's auc: 0.96224	valid_1's auc: 0.93325
[90]	training's auc: 0.965693	valid_1's auc: 0.933461
[100]	training's auc: 0.968595	valid_1's auc: 0.933122
[110]	training's auc: 0.971831	valid_1's auc: 0.932772
[120]	training's auc: 0.974155	valid_1's auc: 0.932426
Early stopping, best iteration is:
[28]	training's auc: 0.940784	valid_1's auc: 0.935647
----------- 1
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.934138	valid_1's auc: 0.933795
[20]	training's auc: 0.937053	valid_1's auc: 0.937847
[30

,name,importance
1453,system_op_days_http_status_code_200_cumsum_woe,52174.994882
1204,system_system_op_times_groups_http_status_code...,44540.777444
825,op_month_ip_transform_cumsum_woe,27863.447757
200,op_times_groups_user_name_url_wpsdoc.xxx.com/d...,17382.354340
730,op_month_device_num_transform_cumsum_woe,5221.451267
509,op_month_ip_transform_os_type_win_cumsum_woe,4257.597877
970,system_system_op_times_groups_ip_type_内网_cumsu...,3910.711483
589,op_times_groups_ip_transform_log_system_transf...,3057.761062
807,system_system_op_times_groups_department_rd_cu...,2602.702286
18,op_days_user_name_ip_type_内网_cumsum_woe,2228.769586


训练集AUC均值：0.9375042875050529, 训练集合AUC方差: 0.0026293447882548165


## 时间序列特征woe编码(category_encoders)+lightgbm

### 时间序列特征woe编码

In [29]:
df_row_train = pd.merge(left=df_train_time_sequence, right=df_raw_train[['id', y_label]], on='id')
df_row_val = pd.merge(left=df_val_time_sequence, right=df_raw_val[['id']], on='id')

In [30]:
cols = df_row_train.columns
cols = cols.drop(['id', y_label])

ord_encoder = ce.WOEEncoder(cols=cols)

df_row_val[y_label] = np.nan
df_train_time_sequence_woe = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
df_val_time_sequence_woe  = ord_encoder.transform(df_row_val)


display(df_train_time_sequence_woe.head())
display(df_val_time_sequence_woe.head())


,id,op_diff_second1,op_diff_second2,system_op_diff_second1,system_op_diff_second2,op_datetime_year,op_datetime_month,op_datetime_day,op_datetime_hour,op_datetime_minute,op_datetime_second,op_datetime_quarter,op_datetime_dayofweek,op_datetime_is_year_start,op_datetime_is_month_start,op_datetime_is_month_end,op_datetime_second_sin,op_datetime_second_cos,op_datetime_minute_sin,op_datetime_minute_cos,op_datetime_hour_sin,op_datetime_hour_cos,op_datetime_day_sin,op_datetime_day_cos,op_datetime_dayofweek_sin,op_datetime_dayofweek_cos,op_datetime_month_sin,op_datetime_month_cos,avg_3_system_op_diff_second1,median_3_system_op_diff_second1,max_3_system_op_diff_second1,min_3_system_op_diff_second1,std_3_system_op_diff_second1,skew_3_system_op_diff_second1,kurt_3_system_op_diff_second1,avg_6_system_op_diff_second1,median_6_system_op_diff_second1,max_6_system_op_diff_second1,min_6_system_op_diff_second1,std_6_system_op_diff_second1,skew_6_system_op_diff_second1,kurt_6_system_op_diff_second1,avg_9_system_op_diff_second1,median_9_system_op_diff_second1,max_9_system_op_diff_second1,min_9_system_op_diff_second1,std_9_system_op_diff_second1,skew_9_system_op_diff_second1,kurt_9_system_op_diff_second1,avg_12_system_op_diff_second1,...,avg_log_system_transform_system_op_diff_second2_12,median_log_system_transform_system_op_diff_second2_12,max_log_system_transform_system_op_diff_second2_12,min_log_system_transform_system_op_diff_second2_12,std_log_system_transform_system_op_diff_second2_12,skew_log_system_transform_system_op_diff_second2_12,kurt_log_system_transform_system_op_diff_second2_12,avg_log_system_transform_system_op_diff_second2_15,median_log_system_transform_system_op_diff_second2_15,max_log_system_transform_system_op_diff_second2_15,min_log_system_transform_system_op_diff_second2_15,std_log_system_transform_system_op_diff_second2_15,skew_log_system_transform_system_op_diff_second2_15,kurt_log_system_transform_system_op_diff_second2_15,avg_url_system_op_diff_second2_3,median_url_system_op_diff_second2_3,max_url_system_op_diff_second2_3,min_url_system_op_diff_second2_3,std_url_system_op_diff_second2_3,skew_url_system_op_diff_second2_3,kurt_url_system_op_diff_second2_3,avg_url_system_op_diff_second2_6,median_url_system_op_diff_second2_6,max_url_system_op_diff_second2_6,min_url_system_op_diff_second2_6,std_url_system_op_diff_second2_6,skew_url_system_op_diff_second2_6,kurt_url_system_op_diff_second2_6,avg_url_system_op_diff_second2_9,median_url_system_op_diff_second2_9,max_url_system_op_diff_second2_9,min_url_system_op_diff_second2_9,std_url_system_op_diff_second2_9,skew_url_system_op_diff_second2_9,kurt_url_system_op_diff_second2_9,avg_url_system_op_diff_second2_12,median_url_system_op_diff_second2_12,max_url_system_op_diff_second2_12,min_url_system_op_diff_second2_12,std_url_system_op_diff_second2_12,skew_url_system_op_diff_second2_12,kurt_url_system_op_diff_second2_12,avg_url_system_op_diff_second2_15,median_url_system_op_diff_second2_15,max_url_system_op_diff_second2_15,min_url_system_op_diff_second2_15,std_url_system_op_diff_second2_15,skew_url_system_op_diff_second2_15,kurt_url_system_op_diff_second2_15,is_risk
0,44477,2.82611,2.255699,0.000000,2.745681,-0.000105,-0.0546,-0.160133,4.819152,0.074188,0.027464,0.003825,-0.137237,-0.000105,0.004134,-0.005843,0.039855,0.039097,0.012152,0.021101,-0.129382,4.810130,-0.160133,-0.188744,-0.137237,-0.122193,-0.0546,-0.0546,3.033363,3.033363,3.033363,3.033363,3.033363,1.244151,-0.000105,3.592979,3.592979,3.592979,3.592979,3.592979,3.305297,3.305297,3.949654,3.949654,3.949654,3.949654,3.949654,3.949654,3.949654,3.438828,...,0.119705,0.119705,0.119705,0.119705,0.119705,0.119705,0.119705,0.11482,0.11482,0.11482,0.11482,0.11482,0.11482,0.11482,1.513537,1.513537,1.513537,1.513537,1.513537,1.416664,-0.000105,0.887963,0.887963,0.887963,0.887963,0.887963,1.051085,1.051085,0.826088,0.826088,0.826088,0.826088,0.826088,0.826088,0.826088,0.697202,0.697202,0.697202,0.697202,0.697202,0.697202,0.697202,0.6464

,id,op_diff_second1,op_diff_second2,system_op_diff_second1,system_op_diff_second2,op_datetime_year,op_datetime_month,op_datetime_day,op_datetime_hour,op_datetime_minute,op_datetime_second,op_datetime_quarter,op_datetime_dayofweek,op_datetime_is_year_start,op_datetime_is_month_start,op_datetime_is_month_end,op_datetime_second_sin,op_datetime_second_cos,op_datetime_minute_sin,op_datetime_minute_cos,op_datetime_hour_sin,op_datetime_hour_cos,op_datetime_day_sin,op_datetime_day_cos,op_datetime_dayofweek_sin,op_datetime_dayofweek_cos,op_datetime_month_sin,op_datetime_month_cos,avg_3_system_op_diff_second1,median_3_system_op_diff_second1,max_3_system_op_diff_second1,min_3_system_op_diff_second1,std_3_system_op_diff_second1,skew_3_system_op_diff_second1,kurt_3_system_op_diff_second1,avg_6_system_op_diff_second1,median_6_system_op_diff_second1,max_6_system_op_diff_second1,min_6_system_op_diff_second1,std_6_system_op_diff_second1,skew_6_system_op_diff_second1,kurt_6_system_op_diff_second1,avg_9_system_op_diff_second1,median_9_system_op_diff_second1,max_9_system_op_diff_second1,min_9_system_op_diff_second1,std_9_system_op_diff_second1,skew_9_system_op_diff_second1,kurt_9_system_op_diff_second1,avg_12_system_op_diff_second1,...,avg_log_system_transform_system_op_diff_second2_12,median_log_system_transform_system_op_diff_second2_12,max_log_system_transform_system_op_diff_second2_12,min_log_system_transform_system_op_diff_second2_12,std_log_system_transform_system_op_diff_second2_12,skew_log_system_transform_system_op_diff_second2_12,kurt_log_system_transform_system_op_diff_second2_12,avg_log_system_transform_system_op_diff_second2_15,median_log_system_transform_system_op_diff_second2_15,max_log_system_transform_system_op_diff_second2_15,min_log_system_transform_system_op_diff_second2_15,std_log_system_transform_system_op_diff_second2_15,skew_log_system_transform_system_op_diff_second2_15,kurt_log_system_transform_system_op_diff_second2_15,avg_url_system_op_diff_second2_3,median_url_system_op_diff_second2_3,max_url_system_op_diff_second2_3,min_url_system_op_diff_second2_3,std_url_system_op_diff_second2_3,skew_url_system_op_diff_second2_3,kurt_url_system_op_diff_second2_3,avg_url_system_op_diff_second2_6,median_url_system_op_diff_second2_6,max_url_system_op_diff_second2_6,min_url_system_op_diff_second2_6,std_url_system_op_diff_second2_6,skew_url_system_op_diff_second2_6,kurt_url_system_op_diff_second2_6,avg_url_system_op_diff_second2_9,median_url_system_op_diff_second2_9,max_url_system_op_diff_second2_9,min_url_system_op_diff_second2_9,std_url_system_op_diff_second2_9,skew_url_system_op_diff_second2_9,kurt_url_system_op_diff_second2_9,avg_url_system_op_diff_second2_12,median_url_system_op_diff_second2_12,max_url_system_op_diff_second2_12,min_url_system_op_diff_second2_12,std_url_system_op_diff_second2_12,skew_url_system_op_diff_second2_12,kurt_url_system_op_diff_second2_12,avg_url_system_op_diff_second2_15,median_url_system_op_diff_second2_15,max_url_system_op_diff_second2_15,min_url_system_op_diff_second2_15,std_url_system_op_diff_second2_15,skew_url_system_op_diff_second2_15,kurt_url_system_op_diff_second2_15,is_risk
0,25002,0.0,0.0,0.000000,0.0,-0.000105,0.0,-0.166861,4.911555,-0.128535,0.067067,-0.010462,1.155995,-0.000105,-0.166861,-0.005843,0.058827,0.059845,-0.117981,-0.063917,1.224615,4.911555,-0.166861,0.001154,1.155995,0.143365,-0.0546,0.0,0.0,1.647069,0.0,0.000000,0.0,0.0,-0.000105,0.0,0.0,3.438828,3.351817,0.0,0.0,0.0,0.0,2.745681,3.438828,3.033363,0.0,0.0,0.0,0.0,...,0.000000,0.000000,3.949654,2.745681,0.0,0.0,0.0,0.0,0.000000,3.949654,-0.230633,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,-0.000105,0.0,0.000000,0.0,3.033363,0.0,0.0,0.0,0.000000,0.000000,0.0,3.033363,0.0,0.0,0.0,0.000000,0.000000,3.949654,2.745681,0.0,0.0,0.0,0.0,0.000000,3.949654,-0.172090,0.0,0.0,0.0,NaN
1,23653,0.0,0.0,1.647069,0.0,-0.000105,0.0,-0.166861,4.911555,0.114251,0.013465,-0.010462,1.155995,-0.000105,-0.166861,-0.005843,0.044126,0.036896,0.071006,0

#### 获取建模使用的特征

In [31]:
remove_cols = ['id']
feats = df_train_time_sequence_woe.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [32]:
df_train = df_train_time_sequence_woe.reset_index(drop=True)
df_test  = df_val_time_sequence_woe.reset_index(drop=True)

#### lightgbm

In [33]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [34]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [35]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.99107	valid_1's auc: 0.987373
[20]	training's auc: 0.995138	valid_1's auc: 0.992062
[30]	training's auc: 0.996641	valid_1's auc: 0.99387
[40]	training's auc: 0.997683	valid_1's auc: 0.995543
[50]	training's auc: 0.998234	valid_1's auc: 0.996243
[60]	training's auc: 0.998718	valid_1's auc: 0.996796
[70]	training's auc: 0.999105	valid_1's auc: 0.997369
[80]	training's auc: 0.999395	valid_1's auc: 0.997779
[90]	training's auc: 0.999639	valid_1's auc: 0.998208
[100]	training's auc: 0.999799	valid_1's auc: 0.998557
[110]	training's auc: 0.999896	valid_1's auc: 0.998785
[120]	training's auc: 0.999949	valid_1's auc: 0.998987
[130]	training's auc: 0.999981	valid_1's auc: 0.999156
[140]	training's auc: 0.999993	valid_1's auc: 0.999273
[150]	training's auc: 0.999998	valid_1's auc: 0.999375
[160]	training's auc: 1	valid_1's auc: 0.999444
[170]	training's auc: 1	valid_1's auc: 0.999486
[180]	training

[390]	training's auc: 1	valid_1's auc: 0.999855
[400]	training's auc: 1	valid_1's auc: 0.999858
[410]	training's auc: 1	valid_1's auc: 0.99986
[420]	training's auc: 1	valid_1's auc: 0.999865
[430]	training's auc: 1	valid_1's auc: 0.999865
[440]	training's auc: 1	valid_1's auc: 0.999865
[450]	training's auc: 1	valid_1's auc: 0.99987
[460]	training's auc: 1	valid_1's auc: 0.999874
[470]	training's auc: 1	valid_1's auc: 0.99988
[480]	training's auc: 1	valid_1's auc: 0.999878
[490]	training's auc: 1	valid_1's auc: 0.999881
[500]	training's auc: 1	valid_1's auc: 0.99988
[510]	training's auc: 1	valid_1's auc: 0.999883
[520]	training's auc: 1	valid_1's auc: 0.999883
[530]	training's auc: 1	valid_1's auc: 0.999888
[540]	training's auc: 1	valid_1's auc: 0.999888
[550]	training's auc: 1	valid_1's auc: 0.99989
[560]	training's auc: 1	valid_1's auc: 0.99989
[570]	training's auc: 1	valid_1's auc: 0.999894
[580]	training's auc: 1	valid_1's auc: 0.999894
[590]	training's auc: 1	valid_1's auc: 0.99989

[1170]	training's auc: 1	valid_1's auc: 0.999897
[1180]	training's auc: 1	valid_1's auc: 0.999898
[1190]	training's auc: 1	valid_1's auc: 0.999898
[1200]	training's auc: 1	valid_1's auc: 0.999898
[1210]	training's auc: 1	valid_1's auc: 0.999899
[1220]	training's auc: 1	valid_1's auc: 0.999899
[1230]	training's auc: 1	valid_1's auc: 0.999899
[1240]	training's auc: 1	valid_1's auc: 0.999899
[1250]	training's auc: 1	valid_1's auc: 0.9999
[1260]	training's auc: 1	valid_1's auc: 0.9999
[1270]	training's auc: 1	valid_1's auc: 0.9999
[1280]	training's auc: 1	valid_1's auc: 0.9999
[1290]	training's auc: 1	valid_1's auc: 0.9999
[1300]	training's auc: 1	valid_1's auc: 0.9999
[1310]	training's auc: 1	valid_1's auc: 0.9999
[1320]	training's auc: 1	valid_1's auc: 0.9999
[1330]	training's auc: 1	valid_1's auc: 0.999901
[1340]	training's auc: 1	valid_1's auc: 0.999901
[1350]	training's auc: 1	valid_1's auc: 0.999901
[1360]	training's auc: 1	valid_1's auc: 0.999901
[1370]	training's auc: 1	valid_1's a

[360]	training's auc: 1	valid_1's auc: 0.999843
[370]	training's auc: 1	valid_1's auc: 0.999851
[380]	training's auc: 1	valid_1's auc: 0.999857
[390]	training's auc: 1	valid_1's auc: 0.999859
[400]	training's auc: 1	valid_1's auc: 0.999864
[410]	training's auc: 1	valid_1's auc: 0.999868
[420]	training's auc: 1	valid_1's auc: 0.99987
[430]	training's auc: 1	valid_1's auc: 0.999874
[440]	training's auc: 1	valid_1's auc: 0.999881
[450]	training's auc: 1	valid_1's auc: 0.999884
[460]	training's auc: 1	valid_1's auc: 0.999888
[470]	training's auc: 1	valid_1's auc: 0.999892
[480]	training's auc: 1	valid_1's auc: 0.999894
[490]	training's auc: 1	valid_1's auc: 0.999896
[500]	training's auc: 1	valid_1's auc: 0.999898
[510]	training's auc: 1	valid_1's auc: 0.999903
[520]	training's auc: 1	valid_1's auc: 0.999904
[530]	training's auc: 1	valid_1's auc: 0.999907
[540]	training's auc: 1	valid_1's auc: 0.999909
[550]	training's auc: 1	valid_1's auc: 0.99991
[560]	training's auc: 1	valid_1's auc: 0.9

,name,importance
36,max_6_system_op_diff_second1,70867.688331
554,max_ip_transform_system_op_diff_second2_3,35847.411862
552,avg_ip_transform_system_op_diff_second2_3,22497.443792
62,avg_ip_transform_system_op_diff_second1_3,13409.695696
351,max_ip_type_system_op_diff_second1_6,8814.737376
615,avg_user_name_system_op_diff_second2_15,7809.957863
580,avg_ip_transform_system_op_diff_second2_15,6807.588831
69,avg_ip_transform_system_op_diff_second1_6,6698.379420
559,avg_ip_transform_system_op_diff_second2_6,6565.253978
895,avg_http_status_code_system_op_diff_second2_15,5267.361330


训练集AUC均值：0.9999014208253214, 训练集合AUC方差: 2.5687274937684798e-05


In [36]:
df_val_time_sequence_woe.head()

,id,op_diff_second1,op_diff_second2,system_op_diff_second1,system_op_diff_second2,op_datetime_year,op_datetime_month,op_datetime_day,op_datetime_hour,op_datetime_minute,op_datetime_second,op_datetime_quarter,op_datetime_dayofweek,op_datetime_is_year_start,op_datetime_is_month_start,op_datetime_is_month_end,op_datetime_second_sin,op_datetime_second_cos,op_datetime_minute_sin,op_datetime_minute_cos,op_datetime_hour_sin,op_datetime_hour_cos,op_datetime_day_sin,op_datetime_day_cos,op_datetime_dayofweek_sin,op_datetime_dayofweek_cos,op_datetime_month_sin,op_datetime_month_cos,avg_3_system_op_diff_second1,median_3_system_op_diff_second1,max_3_system_op_diff_second1,min_3_system_op_diff_second1,std_3_system_op_diff_second1,skew_3_system_op_diff_second1,kurt_3_system_op_diff_second1,avg_6_system_op_diff_second1,median_6_system_op_diff_second1,max_6_system_op_diff_second1,min_6_system_op_diff_second1,std_6_system_op_diff_second1,skew_6_system_op_diff_second1,kurt_6_system_op_diff_second1,avg_9_system_op_diff_second1,median_9_system_op_diff_second1,max_9_system_op_diff_second1,min_9_system_op_diff_second1,std_9_system_op_diff_second1,skew_9_system_op_diff_second1,kurt_9_system_op_diff_second1,avg_12_system_op_diff_second1,...,avg_log_system_transform_system_op_diff_second2_12,median_log_system_transform_system_op_diff_second2_12,max_log_system_transform_system_op_diff_second2_12,min_log_system_transform_system_op_diff_second2_12,std_log_system_transform_system_op_diff_second2_12,skew_log_system_transform_system_op_diff_second2_12,kurt_log_system_transform_system_op_diff_second2_12,avg_log_system_transform_system_op_diff_second2_15,median_log_system_transform_system_op_diff_second2_15,max_log_system_transform_system_op_diff_second2_15,min_log_system_transform_system_op_diff_second2_15,std_log_system_transform_system_op_diff_second2_15,skew_log_system_transform_system_op_diff_second2_15,kurt_log_system_transform_system_op_diff_second2_15,avg_url_system_op_diff_second2_3,median_url_system_op_diff_second2_3,max_url_system_op_diff_second2_3,min_url_system_op_diff_second2_3,std_url_system_op_diff_second2_3,skew_url_system_op_diff_second2_3,kurt_url_system_op_diff_second2_3,avg_url_system_op_diff_second2_6,median_url_system_op_diff_second2_6,max_url_system_op_diff_second2_6,min_url_system_op_diff_second2_6,std_url_system_op_diff_second2_6,skew_url_system_op_diff_second2_6,kurt_url_system_op_diff_second2_6,avg_url_system_op_diff_second2_9,median_url_system_op_diff_second2_9,max_url_system_op_diff_second2_9,min_url_system_op_diff_second2_9,std_url_system_op_diff_second2_9,skew_url_system_op_diff_second2_9,kurt_url_system_op_diff_second2_9,avg_url_system_op_diff_second2_12,median_url_system_op_diff_second2_12,max_url_system_op_diff_second2_12,min_url_system_op_diff_second2_12,std_url_system_op_diff_second2_12,skew_url_system_op_diff_second2_12,kurt_url_system_op_diff_second2_12,avg_url_system_op_diff_second2_15,median_url_system_op_diff_second2_15,max_url_system_op_diff_second2_15,min_url_system_op_diff_second2_15,std_url_system_op_diff_second2_15,skew_url_system_op_diff_second2_15,kurt_url_system_op_diff_second2_15,is_risk
0,25002,0.0,0.0,0.000000,0.0,-0.000105,0.0,-0.166861,4.911555,-0.128535,0.067067,-0.010462,1.155995,-0.000105,-0.166861,-0.005843,0.058827,0.059845,-0.117981,-0.063917,1.224615,4.911555,-0.166861,0.001154,1.155995,0.143365,-0.0546,0.0,0.0,1.647069,0.0,0.000000,0.0,0.0,-0.000105,0.0,0.0,3.438828,3.351817,0.0,0.0,0.0,0.0,2.745681,3.438828,3.033363,0.0,0.0,0.0,0.0,...,0.000000,0.000000,3.949654,2.745681,0.0,0.0,0.0,0.0,0.000000,3.949654,-0.230633,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,-0.000105,0.0,0.000000,0.0,3.033363,0.0,0.0,0.0,0.000000,0.000000,0.0,3.033363,0.0,0.0,0.0,0.000000,0.000000,3.949654,2.745681,0.0,0.0,0.0,0.0,0.000000,3.949654,-0.172090,0.0,0.0,0.0,NaN
1,23653,0.0,0.0,1.647069,0.0,-0.000105,0.0,-0.166861,4.911555,0.114251,0.013465,-0.010462,1.155995,-0.000105,-0.166861,-0.005843,0.044126,0.036896,0.071006,0

In [39]:
df_val_time_sequence_woe['op_datetime_hour'].unique()

array([ 4.91155487,  4.67559063,  4.8191522 ,  4.65698545,  4.47438216,
        4.70733933,  4.48028188,  4.36065892,  0.61680583, -0.37971762,
       -0.3475097 ,  0.77897991, -0.13198428, -0.49208522, -0.39027095,
       -0.37373287, -0.42574819, -0.36313596,  4.28612587,  4.49969997,
       -0.48111487,  4.63366833,  4.67357247, -0.05567711])

### psi

In [42]:
import toad 

psi = toad.metrics.PSI(df_train_time_sequence_woe,df_val_time_sequence_woe)
psi.sort_values(0,ascending=False)

max_device_num_transform_system_op_diff_second2_15    2.903963
max_ip_transform_system_op_diff_second2_15            2.779607
max_user_name_system_op_diff_second2_15               2.692449
max_device_num_transform_system_op_diff_second1_15    2.503602
max_ip_transform_system_op_diff_second1_15            2.413967
                                                        ...   
kurt_http_status_code_system_op_diff_second1_3        0.000000
kurt_user_name_system_op_diff_second2_3               0.000000
kurt_op_city_system_op_diff_second1_3                 0.000000
kurt_3_system_op_diff_second1                         0.000000
is_risk                                               0.000000
Length: 1009, dtype: float64

In [46]:
feats = psi[psi<0.1].index

In [49]:
feats = feats.drop(y_label)

In [50]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.981993	valid_1's auc: 0.979326
[20]	training's auc: 0.987034	valid_1's auc: 0.982046
[30]	training's auc: 0.990554	valid_1's auc: 0.985962
[40]	training's auc: 0.992667	valid_1's auc: 0.988505
[50]	training's auc: 0.995051	valid_1's auc: 0.990723
[60]	training's auc: 0.996694	valid_1's auc: 0.99298
[70]	training's auc: 0.997757	valid_1's auc: 0.994527
[80]	training's auc: 0.998501	valid_1's auc: 0.995632
[90]	training's auc: 0.998998	valid_1's auc: 0.996449
[100]	training's auc: 0.999354	valid_1's auc: 0.997023
[110]	training's auc: 0.99959	valid_1's auc: 0.997461
[120]	training's auc: 0.999749	valid_1's auc: 0.997803
[130]	training's auc: 0.999851	valid_1's auc: 0.998126
[140]	training's auc: 0.999914	valid_1's auc: 0.998347
[150]	training's auc: 0.999951	valid_1's auc: 0.998536
[160]	training's auc: 0.999975	valid_1's auc: 0.998664
[170]	training's auc: 0.999989	valid_1's auc: 0.998766


[410]	training's auc: 1	valid_1's auc: 0.99958
[420]	training's auc: 1	valid_1's auc: 0.999589
[430]	training's auc: 1	valid_1's auc: 0.999595
[440]	training's auc: 1	valid_1's auc: 0.999601
[450]	training's auc: 1	valid_1's auc: 0.999612
[460]	training's auc: 1	valid_1's auc: 0.999616
[470]	training's auc: 1	valid_1's auc: 0.999623
[480]	training's auc: 1	valid_1's auc: 0.999625
[490]	training's auc: 1	valid_1's auc: 0.99963
[500]	training's auc: 1	valid_1's auc: 0.999635
[510]	training's auc: 1	valid_1's auc: 0.999641
[520]	training's auc: 1	valid_1's auc: 0.999641
[530]	training's auc: 1	valid_1's auc: 0.99965
[540]	training's auc: 1	valid_1's auc: 0.999653
[550]	training's auc: 1	valid_1's auc: 0.999659
[560]	training's auc: 1	valid_1's auc: 0.999657
[570]	training's auc: 1	valid_1's auc: 0.999665
[580]	training's auc: 1	valid_1's auc: 0.999671
[590]	training's auc: 1	valid_1's auc: 0.999673
[600]	training's auc: 1	valid_1's auc: 0.999672
[610]	training's auc: 1	valid_1's auc: 0.99

[220]	training's auc: 1	valid_1's auc: 0.999132
[230]	training's auc: 1	valid_1's auc: 0.999168
[240]	training's auc: 1	valid_1's auc: 0.999218
[250]	training's auc: 1	valid_1's auc: 0.999253
[260]	training's auc: 1	valid_1's auc: 0.999283
[270]	training's auc: 1	valid_1's auc: 0.999298
[280]	training's auc: 1	valid_1's auc: 0.999326
[290]	training's auc: 1	valid_1's auc: 0.99935
[300]	training's auc: 1	valid_1's auc: 0.999369
[310]	training's auc: 1	valid_1's auc: 0.999388
[320]	training's auc: 1	valid_1's auc: 0.999407
[330]	training's auc: 1	valid_1's auc: 0.999428
[340]	training's auc: 1	valid_1's auc: 0.999447
[350]	training's auc: 1	valid_1's auc: 0.99946
[360]	training's auc: 1	valid_1's auc: 0.999479
[370]	training's auc: 1	valid_1's auc: 0.999493
[380]	training's auc: 1	valid_1's auc: 0.999499
[390]	training's auc: 1	valid_1's auc: 0.999512
[400]	training's auc: 1	valid_1's auc: 0.999532
[410]	training's auc: 1	valid_1's auc: 0.999537
[420]	training's auc: 1	valid_1's auc: 0.9

[590]	training's auc: 1	valid_1's auc: 0.999671
[600]	training's auc: 1	valid_1's auc: 0.999676
[610]	training's auc: 1	valid_1's auc: 0.999682
[620]	training's auc: 1	valid_1's auc: 0.999685
[630]	training's auc: 1	valid_1's auc: 0.999686
[640]	training's auc: 1	valid_1's auc: 0.99969
[650]	training's auc: 1	valid_1's auc: 0.999695
[660]	training's auc: 1	valid_1's auc: 0.999699
[670]	training's auc: 1	valid_1's auc: 0.999701
[680]	training's auc: 1	valid_1's auc: 0.999704
[690]	training's auc: 1	valid_1's auc: 0.999707
[700]	training's auc: 1	valid_1's auc: 0.999709
[710]	training's auc: 1	valid_1's auc: 0.999711
[720]	training's auc: 1	valid_1's auc: 0.999712
[730]	training's auc: 1	valid_1's auc: 0.999714
[740]	training's auc: 1	valid_1's auc: 0.999717
[750]	training's auc: 1	valid_1's auc: 0.999719
[760]	training's auc: 1	valid_1's auc: 0.99972
[770]	training's auc: 1	valid_1's auc: 0.999723
[780]	training's auc: 1	valid_1's auc: 0.999724
[790]	training's auc: 1	valid_1's auc: 0.9

[940]	training's auc: 1	valid_1's auc: 0.999849
[950]	training's auc: 1	valid_1's auc: 0.999847
[960]	training's auc: 1	valid_1's auc: 0.999852
[970]	training's auc: 1	valid_1's auc: 0.999852
[980]	training's auc: 1	valid_1's auc: 0.999852
[990]	training's auc: 1	valid_1's auc: 0.999852
[1000]	training's auc: 1	valid_1's auc: 0.999852
[1010]	training's auc: 1	valid_1's auc: 0.999852
[1020]	training's auc: 1	valid_1's auc: 0.999852
[1030]	training's auc: 1	valid_1's auc: 0.999854
[1040]	training's auc: 1	valid_1's auc: 0.999854
[1050]	training's auc: 1	valid_1's auc: 0.999855
[1060]	training's auc: 1	valid_1's auc: 0.999855
[1070]	training's auc: 1	valid_1's auc: 0.999856
[1080]	training's auc: 1	valid_1's auc: 0.999855
[1090]	training's auc: 1	valid_1's auc: 0.999856
[1100]	training's auc: 1	valid_1's auc: 0.999856
[1110]	training's auc: 1	valid_1's auc: 0.999856
[1120]	training's auc: 1	valid_1's auc: 0.999858
[1130]	training's auc: 1	valid_1's auc: 0.999857
[1140]	training's auc: 1	v

,name,importance
46,avg_ip_transform_system_op_diff_second1_6,78792.916005
328,avg_ip_transform_system_op_diff_second2_3,41645.469295
4,op_datetime_hour,24967.787574
213,avg_ip_type_system_op_diff_second1_6,6773.415092
530,avg_http_status_code_system_op_diff_second2_15,6466.369409
42,avg_ip_transform_system_op_diff_second1_3,5710.053907
526,avg_http_status_code_system_op_diff_second2_12,4624.472946
25,avg_6_system_op_diff_second1,4262.328164
224,min_ip_type_system_op_diff_second1_15,4173.825762
331,skew_ip_transform_system_op_diff_second2_3,3911.970249


训练集AUC均值：0.9997751275645284, 训练集合AUC方差: 5.5514694160616674e-05


## 类别特征+时间序列特征WOE编码+lightgbm

In [6]:
df_row_train = pd.read_csv(path_train)
df_row_val  = pd.read_csv(path_test)

In [7]:
bins_sp_cate = sp.woebin(df_row_train, y_label, var_skip=['id'])
bins_sp_cate = sp.bins_adj_nan_others(df_row_train, bins_sp_cate)

train_sp_cate_woe = sp.woebin_ply(df_row_train, bins_sp_cate)
val_sp_cate_woe  = sp.woebin_ply(df_row_val, bins_sp_cate)


There are 2 date/time type columns are removed from input dataset. 
 (ColumnNames: op_datetime, op_month)


>>> There are 3 variables have too many unique non-numberic values, which might cause the binning process slow. Please double check the following variables: 
device_num_transform, ip_transform, user_name
>>> Continue the binning process?
1: yes 
2: no 
3: remove these variables
Selection: 3


There are 1 variables have only one binning intervals，please check the binning result. 
 (ColumnNames: ip_type)


[INFO] converting into woe values ...
[INFO] converting into woe values ...


In [11]:
feats = [col for col in train_sp_cate_woe.columns if col[-4:]=='_woe']
save_pkl(train_sp_cate_woe[feats+['id']], path_new_train_sp_cate)
save_pkl(val_sp_cate_woe[feats+['id']], path_new_test_sp_cate)

### lightgbm

In [12]:
df_train = train_sp_cate_woe.reset_index(drop=True)
df_test  = val_sp_cate_woe.reset_index(drop=True)

In [13]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [14]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.682155	valid_1's auc: 0.663911
[20]	training's auc: 0.683416	valid_1's auc: 0.662432
[30]	training's auc: 0.684322	valid_1's auc: 0.662442
[40]	training's auc: 0.685323	valid_1's auc: 0.661239
[50]	training's auc: 0.685782	valid_1's auc: 0.65961
[60]	training's auc: 0.686366	valid_1's auc: 0.659697
[70]	training's auc: 0.686861	valid_1's auc: 0.659764
[80]	training's auc: 0.687396	valid_1's auc: 0.659965
[90]	training's auc: 0.687776	valid_1's auc: 0.659966
[100]	training's auc: 0.68811	valid_1's auc: 0.659389
Early stopping, best iteration is:
[1]	training's auc: 0.675575	valid_1's auc: 0.671606
----------- 1
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.682641	valid_1's auc: 0.663567
[20]	training's auc: 0.683419	valid_1's auc: 0.663708
[30]	training's auc: 0.684544	valid_1's auc: 0.663158
[40]	training's auc: 0.684944	valid_1's auc: 0.663706
[50]	

,name,importance
8,http_status_code_woe,11681.748437
2,department_woe,705.857745
5,url_woe,292.370024
7,op_city_woe,82.304533
0,browser_version_woe,51.016398
1,os_version_woe,39.417580
4,log_system_transform_woe,32.171306
6,browser_woe,22.848209
3,os_type_woe,2.127078
9,ip_type_woe,0.000000


训练集AUC均值：0.6679529519688591, 训练集合AUC方差: 0.0030470249314401626
